In [11]:
from queue import Queue
from threading import Thread
def worker(input_q, output_q):
        print("worker")
        while True:
            if input_q.full():
                break
            frame = input_q.get()
            bFind=detect_objectsbyframe(frame)
#            if bFind:
#                capturephoto(frame,1000)
            output_q.put("hello")
            print('shape =',frame.shape)
            
        print("worker exit")
input_q = Queue(2)  # fps is better if queue is higher but then more lags
output_q = Queue()
for i in range(1):
        t = Thread(target=worker, args=(input_q, output_q))
        t.daemon = True
        t.start()
print("test")
#input_q.put('hello')
        

worker
test


In [16]:
import base64
#import cStringIO
import sys
import tempfile
import time
MODEL_BASE = '/Users/aadebuger/GEXT/github2019/models/research'
MODEL_BASE = '/home/aadebugergf/Ai/github2019/models/research'


sys.path.append(MODEL_BASE)
sys.path.append(MODEL_BASE + '/object_detection')
sys.path.append(MODEL_BASE + '/slim')


import numpy as np
from PIL import Image
from PIL import ImageDraw
import tensorflow as tf
from utils import label_map_util




PATH_TO_CKPT = '/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb'
PATH_TO_LABELS = MODEL_BASE + '/object_detection/data/mscoco_label_map.pbtxt'

content_types = {'jpg': 'image/jpeg',
                 'jpeg': 'image/jpeg',
                 'png': 'image/png'}
extensions = sorted(content_types.keys())


def is_image():
  def _is_image(form, field):
    if not field.data:
      raise ValidationError()
    elif field.data.filename.split('.')[-1].lower() not in extensions:
      raise ValidationError()

  return _is_image





class ObjectDetector(object):

  def __init__(self):
    self.detection_graph = self._build_graph()
    self.sess = tf.Session(graph=self.detection_graph)

    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    self.category_index = label_map_util.create_category_index(categories)

  def _build_graph(self):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    return detection_graph

  def _load_image_into_numpy_array(self, image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

  def detect(self, image):
    image_np = self._load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    graph = self.detection_graph
    image_tensor = graph.get_tensor_by_name('image_tensor:0')
    boxes = graph.get_tensor_by_name('detection_boxes:0')
    scores = graph.get_tensor_by_name('detection_scores:0')
    classes = graph.get_tensor_by_name('detection_classes:0')
    num_detections = graph.get_tensor_by_name('num_detections:0')

    (boxes, scores, classes, num_detections) = self.sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    boxes, scores, classes, num_detections = map(
        np.squeeze, [boxes, scores, classes, num_detections])

    return boxes, scores, classes.astype(int), num_detections


def draw_bounding_box_on_image(image, box, color='red', thickness=4):
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  print("image size ",image.size)
  ymin, xmin, ymax, xmax = box
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  print("enlarge =",left*4/3,right*4/3,top*4/3,bottom*4/3)
  print("rect,width="+str(right-left)+"height="+str(bottom-top))
  centerx=left+(right-left)/2
  centery=top+ (bottom-top)/2
  print("center=",centerx,"y=",centery)
  scaraxy=computeposition(centerx*4/3,centery*4/3)
  print("scara xy",scaraxy)
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  print('(left, right, top, bottom)',(left, right, top, bottom))

from io import BytesIO
def encode_image(image):
  image_buffer = BytesIO()
  image.save(image_buffer, format='PNG')
  imagevalue = image_buffer.getvalue()
  base64str=base64.b64encode(imagevalue).decode()

  imgstr = 'data:image/png;base64,{:s}'.format(
     base64str )
  return imgstr


def detect_objects(image_path):
  start = time.time()
  
  image = Image.open(image_path).convert('RGB')
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
  print(num_detections)
  new_images = {}
  for i in range(int(num_detections)):
    print("scres[i",scores[i])
    if scores[i] < 0.35: continue
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)

  result = {}
#  result['original'] = encode_image(image.copy())
  print(new_images)
  count = 0 
  for cls, new_image in new_images.items():
    category = client.category_index[cls]['name']
    print(category)
    result[category] = encode_image(new_image)
    new_image.save("{0}.jpg".format(count))
    count=count+1
  end = time.time()
  print("execute2 time",(end-start)) 
  return result


def computeposition(x,y):
    yuandian=(853.5124299065,706.912679109)
    kx = -0.8937383178 
    ky = -0.8729680915
    scarax = yuandian[0] + kx*x 
    scaray = yuandian[1] + ky*y
    return (scarax,scaray)

def detect_objectsbyframe(frame):
  start = time.time()
  
#  image = Image.open(image_path).convert('RGB')
  height, width, channels = frame.shape
  image= Image.frombytes("RGB", (width,height), frame.tostring())
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
#  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
#  print(num_detections)
  new_images = {}
  bFind = False
  for i in range(int(num_detections)):
    print("scres[i",scores[i])
    if scores[i] < 0.35: continue
    bFind=True
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)

  result = {}
#  result['original'] = encode_image(image.copy())
  print(new_images)
  count = 0 
  for cls, new_image in new_images.items():
    category = client.category_index[cls]['name']
    print(category)
    result[category] = encode_image(new_image)
    new_image.save("{0}.jpg".format(count))
    count=count+1
  end = time.time()
#  print("execute2 time",(end-start)) 
#  return result
   return bFind 
PATH_TO_CKPT='/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'
PATH_TO_CKPT='/home/aadebugergf/Ai/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'

modelname='ssd_inception_v2_coco_2018_01_28'
PATH_TO_CKPT='/home/aadebugergf/Ai/models/'+modelname+'/frozen_inference_graph.pb'

client = ObjectDetector()

IndentationError: unexpected indent (<ipython-input-16-d9dd0d835220>, line 206)

In [17]:
import ipywidgets as widgets
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
cap = cv2.VideoCapture(1)
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)

def videooutput():
    global ivalue
    bFirst=False
    count = 0 
    while(True):
        ret, frame = cap.read()
        if bFirst is False:
            input_q.put(frame)
            bFirst = True
        if output_q.empty():
                pass
        else:
                data=output_q.get()
                bFirst=False
        img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()

        if ( (end-start)>60):
            break
videooutput()
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)


{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
scres[i 0.51486236
boxes [0.71972424 0.05361131 1.         0.9859897 ]
image size  (480, 360)
enlarge = 34.31123733520508 631.0334014892578 345.4676342010498 480.0
rect,width=447.54162311553955height=100.89927434921265
center= 249.50423955917358 y= 309.5503628253937
scara xy (556.190430776388, 346.6092264972447)
(left, right, top, bottom) (25.73342800140381, 473.27505111694336, 259.10072565078735, 360.0)
{5: <PIL.Image.Image image mode=RGB size=480x360 at 0x7FBDC25617B8>}
airplane
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 640, 3)
{}
shape = (480, 

In [3]:
cap.release()